# Dataset & preparation

In [1]:
from scipy.io import loadmat
ff=loadmat("./MNISTData.mat")

import time
fexport=open("./statistics.MNIST.pytorch.export.csv","w+")
print("Dataset,Subnet 1,Subnet 2,Loss function,Accuracy,Step,Momentum,Batch,Epoch",file=fexport)


In [2]:
import torch

data_train=torch.zeros(60000,28,28)
data_test=torch.zeros(10000,28,28)
label_train=torch.zeros(60000,10)
label_test=torch.zeros(10000,10)

for i in range(60000):
    data_train[i]=torch.from_numpy(ff['X_Train'][:,:,i])
    label_train[i]=torch.from_numpy(ff['D_Train'][:,i])
    
for i in range(10000):
    data_test[i]=torch.from_numpy(ff['X_Test'][:,:,i])
    label_test[i]=torch.from_numpy(ff['D_Test'][:,i])
    
# label_train=label_train.long()

The code block above reformat MatLab variables into python variables.

> Attention: a three-dimension argument imported from matlab differs from a common three-dimension array. A common three-dimension array segaments the elements it stores from macro to micro, whereas that from matlab in a format of row, colume, matrices' order.

In [3]:
# this is a test

footnote=0
import matplotlib.pyplot as plt
plt.imshow(data_train[footnote],cmap='gray')
plt.show()
print(label_train[footnote])

<Figure size 640x480 with 1 Axes>

tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])


# Model

In [4]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 20, 9)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(2000, 100)  # 6*6 from image dimension
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# Test & Train

In [5]:
for batch in [5,20,50,100]:
    for step in [0.01,0.1,0.5,0.7]:
#         for mm  in [0,0.1,0.3,0.5,0.7,0.9]:
        for mm  in [0]:
            net=Net()
            # create your optimizer
            optimizer = optim.SGD(net.parameters(), lr=step, momentum=mm)
            optimizer.zero_grad()
            criterion = nn.MSELoss()

            for epoch in range(15):
                print('the present epoch is %d' % (epoch+1) )
                start_time = time.time()

                for j in range(int(len(data_train)/batch)):
                    optimizer.zero_grad()   # zero the gradient buffers
                    for k in range(batch):
                        i=k+j*batch
                        input=torch.ones(1,1,28,28)
                        input[0,0,:,:]=data_train[i]
                        output = net(input)
                        loss = criterion(output, label_train[i].view(1,-1))/batch
                        loss.backward()
                    optimizer.step()    # Does the update

                print("--- the last epoch: %s seconds ---" % (time.time() - start_time))

                accuracy=0
                for i in range(10000):
                    input=torch.ones(1,1,28,28)
                    input[0,0,:,:]=data_test[i]
                    output=torch.round(net(input))
                    if torch.sum(criterion(output, label_test[i].view(1,-1)))<=0.01:
                        accuracy+=1
                print("The accuracy for persent epoch is ", accuracy/10000)
                print("MNIST,","shallow,","shallow,","MSE,",accuracy/10000,",",step,",",mm,",",batch,",",epoch+1,file=fexport)
                

the present epoch is 1


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


--- the last epoch: 79.12769222259521 seconds ---
The accuracy for persent epoch is  0.7684
the present epoch is 2
--- the last epoch: 76.8226010799408 seconds ---
The accuracy for persent epoch is  0.882
the present epoch is 3
--- the last epoch: 76.86796689033508 seconds ---
The accuracy for persent epoch is  0.9011
the present epoch is 4
--- the last epoch: 76.66919493675232 seconds ---
The accuracy for persent epoch is  0.9155
the present epoch is 5
--- the last epoch: 76.65495014190674 seconds ---
The accuracy for persent epoch is  0.9266
the present epoch is 6
--- the last epoch: 76.2812089920044 seconds ---
The accuracy for persent epoch is  0.9341
the present epoch is 7
--- the last epoch: 76.10788488388062 seconds ---
The accuracy for persent epoch is  0.9417
the present epoch is 8
--- the last epoch: 78.24752283096313 seconds ---
The accuracy for persent epoch is  0.9494
the present epoch is 9
--- the last epoch: 75.968101978302 seconds ---
The accuracy for persent epoch is  

The accuracy for persent epoch is  0.9041
the present epoch is 13
--- the last epoch: 79.80946707725525 seconds ---
The accuracy for persent epoch is  0.908
the present epoch is 14
--- the last epoch: 80.83792281150818 seconds ---
The accuracy for persent epoch is  0.9114
the present epoch is 15
--- the last epoch: 79.7111439704895 seconds ---
The accuracy for persent epoch is  0.9156
the present epoch is 1
--- the last epoch: 80.05017399787903 seconds ---
The accuracy for persent epoch is  0.8788
the present epoch is 2
--- the last epoch: 78.85597109794617 seconds ---
The accuracy for persent epoch is  0.925
the present epoch is 3
--- the last epoch: 79.4271969795227 seconds ---
The accuracy for persent epoch is  0.9452
the present epoch is 4
--- the last epoch: 81.22098994255066 seconds ---
The accuracy for persent epoch is  0.9577
the present epoch is 5
--- the last epoch: 84.84998512268066 seconds ---
The accuracy for persent epoch is  0.9662
the present epoch is 6
--- the last epo

--- the last epoch: 80.20321416854858 seconds ---
The accuracy for persent epoch is  0.9582
the present epoch is 10
--- the last epoch: 84.13126111030579 seconds ---
The accuracy for persent epoch is  0.9619
the present epoch is 11
--- the last epoch: 79.22689390182495 seconds ---
The accuracy for persent epoch is  0.9644
the present epoch is 12
--- the last epoch: 89.00552105903625 seconds ---
The accuracy for persent epoch is  0.9659
the present epoch is 13
--- the last epoch: 79.10393381118774 seconds ---
The accuracy for persent epoch is  0.9674
the present epoch is 14
--- the last epoch: 81.8384518623352 seconds ---
The accuracy for persent epoch is  0.9694
the present epoch is 15
--- the last epoch: 85.47864198684692 seconds ---
The accuracy for persent epoch is  0.9707
the present epoch is 1
--- the last epoch: 77.14030432701111 seconds ---
The accuracy for persent epoch is  0.9205
the present epoch is 2
--- the last epoch: 82.60592317581177 seconds ---
The accuracy for persent 

The accuracy for persent epoch is  0.9638
the present epoch is 6
--- the last epoch: 74.08349800109863 seconds ---
The accuracy for persent epoch is  0.9692
the present epoch is 7
--- the last epoch: 74.34796595573425 seconds ---
The accuracy for persent epoch is  0.973
the present epoch is 8
--- the last epoch: 74.68879985809326 seconds ---
The accuracy for persent epoch is  0.975
the present epoch is 9
--- the last epoch: 74.12976098060608 seconds ---
The accuracy for persent epoch is  0.9766
the present epoch is 10
--- the last epoch: 74.05074405670166 seconds ---
The accuracy for persent epoch is  0.9778
the present epoch is 11
--- the last epoch: 74.28089809417725 seconds ---
The accuracy for persent epoch is  0.9787
the present epoch is 12
--- the last epoch: 74.38801383972168 seconds ---
The accuracy for persent epoch is  0.9803
the present epoch is 13
--- the last epoch: 73.93651723861694 seconds ---
The accuracy for persent epoch is  0.9812
the present epoch is 14
--- the last

In [6]:
fexport.close()